In [ ]:
import numpy as np
import pyemma
import pyemma.datasets
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# 1 The connectivity problem in Markov state models

When estimating conventional Markov state models from a set of many short trajectories, one is faced with the problem of finding the "connected set" or "ergodic set". This is the set of Markov states for which a stationary distribution can be computed. It consits of all the Markov states that can be exited and entered an unlimited number of times when following transition that have non-zero probability in the transtion matrix. 

Due to ubiquitous barrier recrossing (also called projection error) it is unsually not possible to decide if a given transition probability exactly zero or non-zero. This renders the identification of the connected set difficult. 
While this is typically not a problem when analysing a single long trajectory, a set of short trajectories may contain a high number of potentially absorbing states (that are never exited). In the latter setting, proper identification of these absorbing states is indispensable to compute correct estimates of free energ differences and kinetics.

Here I demonstrate the problem with a toy model and propose two methods for its solution.



I start, by producing some data that is by construction useless for the estimation of an MSM.
I simulate a couple of short trajectories in the two wells of a double-well toy energy model and prevent any well-to-well transtion from happing by intterupting the simulation before any such transition is about to take place.

In [ ]:
np.random.seed(7777)

In [ ]:
data = pyemma.datasets.load_2well_discrete()
fel = -np.log(data.msm.stationary_distribution)
plt.plot(fel)
plt.ylabel('energy')
plt.xlabel('x')
#plt.plot([48, 48], [2, 25], 'r', label='stop here!')
#plt.plot([52, 52], [2, 25], 'r', label='stop there!')
plt.annotate('interrupt here', xy=(48, fel[48]), xytext=(48-18, fel[48]+1), arrowprops=dict(facecolor='red'))
plt.annotate('interrupt here', xy=(52, fel[52]), xytext=(52+5, fel[52]+1), arrowprops=dict(facecolor='red'))
plt.ylim((3, 10))
plt.xlim((20, 80))
plt.legend()

To make the example more realistic, I add some uncorrelated noise to the trajectories.
In a real molecular system this noise could originate from an uncorrelated (orthogonal) degree of freedom whose amplitude that was insufficiently suppressed by the non-ideal choice of features and dimensionality reduction.

In [ ]:
# simulating non-connected data
trajs1 = [ data.generate_traj(10000, start=30, stop=48).astype(float) for _ in range(4) ]
trajs2 = [ data.generate_traj(10000, start=70, stop=52).astype(float) for _ in range(2) ]
trajs = trajs1 + trajs2
trajs = [ t + 3*np.random.randn(len(t)) for t in trajs ] # add some noise to model a second,
# non-metastable dimension that is not completely orthogonal to the x-coordinate of the double-well

In [ ]:
plt.plot(trajs[0])
plt.plot(trajs[-1])
plt.xlabel('time / steps')
plt.ylabel('x')

I proceed with *k*-means clustering and state of the art MSM estimation.

In [ ]:
# use some default algorithm
kmeans = pyemma.coordinates.cluster_kmeans(data=trajs, k=101, fixed_seed=True)
dtrajs = kmeans.dtrajs

In [ ]:
cumlength = sum([len(d) for d in dtrajs])
print('commulative length of all data is', cumlength)

In [ ]:
# Compute the implied time scales plot for the MSM
its = pyemma.msm.its(dtrajs)
pyemma.plots.plot_implied_timescales(its)
plt.plot([0, its.lagtimes[-1]], [cumlength, cumlength], 'd-', color='black', label='data length') # total data
true_its = data.msm.timescales()[0]
plt.plot([0, its.lagtimes[-1]], [true_its, true_its], 'd-',  color='red', label='true value') 
plt.legend()

Apparently there is aprocess that is much slower than the total length of the simulations data.
Does that mean that the MSM just did its magic and that we have a result? 
No, since by construction, in this example the
relevant well-to-well transition has not been sampled!
So the time scale is an artifact.
One also sees that the ITS it not converging (over the years, standards have become lower and lower, so some people might call this "convergence" but acutually it's not).

This artifact has also an influence on the free-energies (and stationary probabilities) that can be computed form an MSM. 
To see this, I plot the stationary probability of the left well as a function of MSM lag time.

In [ ]:
# Now monitor the convergence of the free energy 
left_well = np.where(kmeans.clustercenters < 50)[0]
p_left = []
for m in its.models:
    pi_full = np.zeros(kmeans.clustercenters.shape[0])
    pi_full[m.active_set] = m.pi
    p_left.append(
        pi_full[left_well].sum()
    )
plt.plot(its.lags, p_left, label='MSM estimate')
plt.plot([0, its.lagtimes[-1]], [0.5, 0.5], color='black', label='true value') # true value
plt.ylabel('P(left well)')
plt.xlabel('lag time / steps')
plt.legend()


Note that we get some results but they are completely arbitrary. These values are essentially random.

The CK-test doesn't tell us that there is a problem with our model.
From this CK-test we mostly learn that the system is very metastable and 
that this metastability is reproduced accross many choices of the lag time.
But we don't learn much about the presence or absence of transitions.

In [ ]:
msm = pyemma.msm.estimate_markov_model(dtrajs, 100)
cktest = msm.cktest(nsets=2)
pyemma.plots.plot_cktest(cktest);

# 2. Ways to solve the connectivity problem

# 2.1 hidden Markov models

HMM can solve the problem of spurious connectivty using the additional layer of stochstic modelling that is built into the definition of an HMM. In HMMs, the trajectory is modelled as a sequence of transitions in some hidden space plus instantaneous emissions into the observed space. A state-to-state transition in the observed space does not need to correspond to a transition in the hidden space. It could just be that the system remains in the same hidden space but two successive "emissions" into the observed space just happen to fall into two different observed states.

In principle, HMMs can deconvolute the simulation data into a) the "actual" transitions in the hidden space and b) a map between that hidden space and the observed space that can explain the the "overlap" of hidden states when viewed from the observed space. 

In [ ]:
hmm = pyemma.msm.estimate_hidden_markov_model(dtrajs, lag=10, nstates=2)

In [ ]:
hmm.count_matrix_EM

The HMM tells us that the number of transtions between the *hidden* states is around 10^-10 which is close to the numerical procision of double precision numbers. This points towards the fact that these transitons might actually
not be there.

In practice, the ability of HMM to identify this probelm can depend on many parameters like the lag-time and the clustering (I suspect that one needs to have more than one cluster center per hidden state to be able to indentify connectivity issues with HMMs).

In [ ]:
hmm.timescales()

Still the HMM sometimes reports an artifactual time scale and there is not good way of telling whether this value is
and artifact or not, without prior knowledge (and without looking at `count_matrix_EM`).
The behavior depends on the clustering and on the lag time. Sometimes an artifactual time scale is reported and sometimes not.

# 2.2 core set Markov models

In core set MSM one works with a relatively low number of states (cores). Typically, these cores are metastable on timescales that are comparable with the lag time of the MSM. In contrast to normal MSMs, the conformational space is not fully partitioned into states. Conformations, that can not unambigously assigned to a metastable core, stay "unassigned". In core set MSM transitions have to be counted differently than in normal MSMs. Only when a trajectory goes form one core to a different core, this is treated as a transition. Transitions between a core and the unassigned area that surrounds the core are not counted.

One of the most important prequisites for core set MSMs is a good definition of cores. Luckily, many methods like TICA or VAMP have been developed to detect order parameters for the slowly-autocorrelated dynamic processes. Once the order parameters have been found, it's only a small step to a good definition of metastable states.

I want to build the core MSM directy on the TICA space without going through the procedure of builing a normal MSM
as an intermediate step. The advantages of this approach will become clearer later.

In [ ]:
# In a molecular system we would start the analysis by selecting some features and doing TICA (or VAMP).
# Since the example here is 1-D, I just define some Gaussians on that 1-D axis and user them as features.
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
coordinates = []
for traj in trajs:
    x = traj
    coordinates.append(
        np.array(
            [gaussian(x, 20, 5), gaussian(x, 25, 5),
             gaussian(x, 30, 5), gaussian(x, 35, 5),
             gaussian(x, 40, 5), gaussian(x, 45, 5),
             gaussian(x, 50, 5), gaussian(x, 55, 5),
             gaussian(x, 60, 5), gaussian(x, 65, 5),
             gaussian(x, 70, 5)]).T
    )

In [ ]:
tica = pyemma.coordinates.tica(data=coordinates, lag=100)
tics = tica.get_output(dimensions=np.arange(1))

One sees in the plot below that the first TIC is a good order parameter that separates the two minima (which is not really a hard problem in this 1-D example).

In [ ]:
for tic in tics:
    plt.plot(tic);
plt.xlabel('time / steps')
plt.ylabel('IC1')

In [ ]:
plt.hist(np.concatenate(tics), bins=50);
plt.xlabel('IC1')
plt.ylabel('counts')

The TICA eigenfunctions (together with their respective eigenvalues) can be used to define a so-called kinetic map, which is a coordinate system in which distances are related to the time the molecular system needs to change conformations. [#Noe:JChemTheoryComput:15] The most distant points on the kinetic map correspond to conformations that interconvert the least frequently. For metastable systems, it was observed that most conformations tend to cluster near the N most distant points in the N-dimensional space defined by the kinetic map. [#Weber:ZIB:04] This observation is the motivation of the inner simplex sub-algorithm of the PCCA algorithm. [#Weber:ZIB:02]

### The inner-simplex sub-algorithm of PCCA
The inner-simplex algorithm approximately finds the N most distant points in the N-dimensional space spanned by the dominant eigenfunctions. [#Weber:ZIB:02] The first eigenfunction of the transfer operator is a constant function, which we disregard for computational efficiency. This leaves all distances unchanged. We therefore seek for the N most distant points in a (N-1) dimensional space which is equivalent to finding the vertices of an N-dimensional simplex.

In the initialization stage of the inner simplex algorithm, we identify the pair of most distant points. For a data set consisting of very large number of points $(>10^{6})$, this becomes computationally infeasible when using only pairwise comparisons. We therefore determine the most distant pair heuristically. To this, we first determine all points that lie on the hyper-surface of the axes-parallel bounding box of all data points. Among this subset, which can not contain more than $2\cdot N-2$ points, we pick the most distant pair $(\mathbf{v}_{1},\,\mathbf{v}_{2})$. These two points are the first and second of totally N vertices.

In the remaining stages of the algorithm, we follow the procedure from reference [#Weber:ZIB:02]. In stage two, for every point $\mathbf{x}$, we determine the distance to the line segment $[\mathbf{v}_{1},\,\mathbf{v}_{2}]$ and select the point with the largest distance as the third vertex $\mathbf{v}_{3}$. In the remaining stages, we proceed analogously in higher dimensions and compute for every point the distance to the affine subspace spanned by the already found vertices. The point with maximum distance is added to the set of vertices. This algorithm terminates when N vertices have been found, since the affine space spanned by all N vertices is equal to the space spanned by all data points.

The following example uses the inner_simplex algorithm, that is not yet part of PyEmma and which I have 
implemented here: https://github.com/fabian-paul/simplex

Please contact me on Github to get access to the repo.

In [ ]:
# First, we find the two most distant points in the 1-D TICA space. 
# This is trivial here but the following function implements the general N-dimensional case.
import simplex
vertices = simplex.find_vertices_inner_simplex(tics)
# in general the algorithm has to do N passes over the date where N is the dimension of the TICA space

### Computing metastable memberships

In the next step, we want to label all data points according to their degree of belonging to one of the vertices. A mathematically consistent way to define this degree of belonging is to use barycentric coordinates defined with respect to the vertices $\{\mathbf{v}_{1},\,\mathbf{v}_{2},\ldots,\mathbf{v}_{N}\}$. That is we seek to express the (Cartesian) coordinates of every point in TICA space $\mathbf{x}$ as $\mathbf{x}=\sum_{i}^{N}m_{i}\mathbf{v}_{i}$ where $\sum_{i}^{N}m_{i}=1$. [#Weber:ZIB:02] If we assume that the data points form clusters near the vertices (see below), we can interpret every coefficient $m_{i}$ as the membership of the point $\mathbf{x}$ in the metastable state $i$. A point belongs in majority to metastable state $i$, if $m_{i}\geq\tfrac{1}{2}$. 

In contrast to the *k*-means algorithm, which can also be used to partition the data points, and which is the default choice in MSM construction, [#Prinz:JChemPhys:11] the inner simplex algorithm can also find metastable states that have low frequency in the simulation data. This is important for MD data that consist of many short trajectories, because the frequency of a metastable state in the data might not be representative of the Boltzmann weight of a state. Therefore the partitioning algorithm should not be too reliant on frequencies. Furthermore, sensible memberships that reflect the degree of metastability can not be defined in such a simple manner for *k*-means clusters.

After having identified the metastable states in the conformational space, we next characterize the transitions between the states.

### Defining the core sets
We define the cores as high-membership regions using the metastable memberships computed with PCCA (see section above). That is 
$$S_{i}:=\{\mathbf{x}(t)\mid i=\mathrm{argmax}_{j}m_{j}(t)\text{ and }m_{i}(t)\geq f\}$$ where $f$ is some value between $\tfrac{1}{2}$ and 1. If $f=\tfrac{1}{2}$, two cores $S_{i}$ and $S_{j}$ $(i\neq j)$ may share a common boundary. If $f>\tfrac{1}{2}$, the transition regions between the cores are not assigned to any core.

Note that the variant of the PCCA algorithm used in this work does not guarantee that all metastable memberships are between 0 and 1. Still, our core definition remains practical. Points with $m_{i}>1$ with respect to a metastable state $i$ are just grouped together with points with $m_{i}>f$. Negative memberships are ignored by construction.

###  Computing the transition matrix
The correct definition of the transition matrix $\mathbf{T}(\tau)$ for core-based MSMs is somehow nuanced and different algorithms have been proposed for its estimation from time series data. Here, we use the transition-based assignment (TBA) method that was proposed by Buchete et al. [#Buchete:JPhysChemB:08] The main advantage of this method over the methods from Schütte et al. [#Schutte:JChemPhys:11, #Schuette:EurPhysJSpecTop:12] is that it guarantees that the elements of the resulting transition matrix are probabilities and that its eigenvalues are not larger than one. These properties will be essential for MSM validation. Transition matrix estimation with the TBA method consists of three steps. 

In the first step, we compute metastable memberships $\mathbf{m}(t)=[m_{1}(t),\,\ldots,\,m_{N}(t)]^{\top}$ with respect to all metastable states for all conformations. 

In the second step, we assign every conformation to a core, if possible. That is, we construct a time series of core labels $\{s(t)\}_{t}$ where $s(t)=i$ if and only if $\mathbf{x}(t)\in S_{i}$. Labels $s(t)$ for the remaining conformations that are not part of a core are determined based on the history of visited cores, by splitting transitions at the midpoint. More precisely, for every time step $t$ we find the most recent valid core label $s(t_{-})$ from the past $t_{-}<t$ and the most proximate valid core label $s(t_{+})$ in the future $t_{+}>t$. Then we set $s(t):=s(t_{-})$ if $t-t_{-}<t_{+}-t$ or $s(t):=s(t_{+})$ otherwise. [#Buchete:JPhysChemB:08] The initial and final time steps of a trajectory may stay unassigned in this procedure and we ignore them from now on. 

In the third step, the transition matrix $\mathbf{T}(\tau)$ is estimated form the time series of core labels $\{s(t)\}_{t}$ using standard methods. 

In [ ]:
# b) comute core assignments
ctrajs = simplex.core_assignments(tics, vertices=vertices, f=0.65, d=0.1)

The next plot shows the core label over time for all trajectories. Still, the trajectories show transitions between the two cores which we know here to be artifats. This problem is addressed below.

In [ ]:
# show core-to-core trajectories
for c in ctrajs:
    plt.plot(np.where(c>=0, c, np.nan))
plt.ylim((-0.1,1.1))
# sometimes there are some recorssing events left, despite the introduction of cores 
plt.xlabel('time / step')
plt.ylabel('core index')

Because we were looking for metastable states, we can be almost sure that the fast transitons, where the trajectory switches form one core to another and then immediately returns are due to an imperfect assignment of conformations to cores. Fast switching of cores is incompatible with the metastable nature of the cores.
Therefore we can safely delete such fast switching events from the trajectories.

In [ ]:
# c) We filter out very short visits to core. 
# Technically we relabel the corresponding trajectory pieces as 'unassigned'.
# In a later step, we could re-assign these peices to a core, if we have to.
ctrajs_metastable = simplex.filter_out_short_core_visits(ctrajs, cutoff=10) 
# usually a small life time cutoff is enough. This number should not be larger than the lag time.
# Here I take 1/10 th of the TICA lag time.
for c in ctrajs_metastable:
    plt.plot(np.where(c>=0, c, np.nan))
plt.ylim((-0.1,1.1))
plt.xlabel('time / step')
plt.ylabel('core index')

Indeed no artifactual transition remains. This is confirmed when we compute the count matrix:

In [ ]:
# d) now that we are more or less sure about the discretization quality, compute count matrices.
simplex.milestoning_count_matrix(ctrajs_metastable, lag=1)

In [ ]:
# compare to the HMM count matrix (state indiced might be switched):
hmm.count_matrix.astype(int)

## 2.3 Future work
In future work I will combinine TICA and HMMs in one joint stochastically optimal estimator. That estimator should be ideal for identifying connectivitiy problems by combinding the strength of good order parameters (that are found by the TICA part) and the strenght of HMM that allows to decouple the observed space from hidden space.

Core set MSM have the disadvantage that the user needs to select a cut-off for the core size. Even though the value of the cut-off is approximately universal (a cutoff of 0.55 to 0.75 on the memberships should work is most situations), it would be better, if there were no user-specified parameter at all. HMMs only need to parameters: the number of states and the lag time and are therefore a good starting point for the development of better estimators.